In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

In [2]:
# Create Spark Session
spark  = SparkSession.builder.appName("test").getOrCreate()

In [3]:
# Load Data in Spark
data = spark.read.csv('data.csv', header = True, inferSchema = True)


In [4]:
# Show data
data.show(5)

+-------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+----+----------+-----------+----------+
|         date|Usage_kWh|Lagging_Current_Reactive_Power_kVarh|Leading_Current_Reactive_Power_kVarh|CO2(tCO2)|Lagging_Current_Power_Factor|Leading_Current_Power_Factor| NSM|WeekStatus|Day_of_week| Load_Type|
+-------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+----+----------+-----------+----------+
|1/1/2018 0:15|     3.17|                                2.95|                                 0.0|      0.0|                       73.21|                       100.0| 900|   Weekday|     Monday|Light_Load|
|1/1/2018 0:30|      4.0|                                4.46|                                 0.0|      0.0|                       66.77|                       100.0|1800|

In [24]:
# Show Columns of Data
data.columns

['date',
 'Usage_kWh',
 'Lagging_Current_Reactive_Power_kVarh',
 'Leading_Current_Reactive_Power_kVarh',
 'CO2(tCO2)',
 'Lagging_Current_Power_Factor',
 'Leading_Current_Power_Factor',
 'NSM',
 'WeekStatus',
 'Day_of_week',
 'Load_Type']

In [6]:
# Check null counts of columns
null_counts = {col: data.filter(data[col].isNull()).count() for col in data.columns}
print(null_counts)

{'date': 0, 'Usage_kWh': 0, 'Lagging_Current_Reactive_Power_kVarh': 0, 'Leading_Current_Reactive_Power_kVarh': 0, 'CO2(tCO2)': 0, 'Lagging_Current_Power_Factor': 0, 'Leading_Current_Power_Factor': 0, 'NSM': 0, 'WeekStatus': 0, 'Day_of_week': 0, 'Load_Type': 0}


In [7]:
# Drop Duplicate Records
data = data.dropDuplicates()

In [8]:
data.show(5)

+--------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+-----+----------+-----------+------------+
|          date|Usage_kWh|Lagging_Current_Reactive_Power_kVarh|Leading_Current_Reactive_Power_kVarh|CO2(tCO2)|Lagging_Current_Power_Factor|Leading_Current_Power_Factor|  NSM|WeekStatus|Day_of_week|   Load_Type|
+--------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+-----+----------+-----------+------------+
|2/1/2018 18:45|    40.25|                                8.82|                                 0.5|      0.0|                       97.68|                       99.99|67500|   Weekday|    Tuesday|Maximum_Load|
|5/1/2018 13:00|     8.21|                                 0.0|                               25.81|      0.0|                       100.0|                 

In [9]:
# Convert categorical values to numeric values

# Convert WeekStatus
data = (data.withColumn("WeekStatus", when(col("WeekStatus") == 'Weekday', 0).when(col("WeekStatus") == 'Weekend', 1)))

# Convert Load_Type
data = (data.withColumn("Load_Type", when(col("Load_Type") == 'Light_Load', 0)
.when(col("Load_Type") == 'Medium_Load', 1)
.when(col("Load_Type") == 'Maximum_Load', 2)))


# Convert Day_of_week
data = (data.withColumn("Day_of_week", when(col("Day_of_week") == 'Sunday', 0)
.when(col("Day_of_week") == 'Monday', 1)
.when(col("Day_of_week") == 'Tuesday', 2)
.when(col("Day_of_week") == 'Wednesday', 3)
.when(col("Day_of_week") == 'Thursday', 4)
.when(col("Day_of_week") == 'Friday', 5)
.when(col("Day_of_week") == 'Saturday', 6)
))

data.show(5)

+--------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+-----+----------+-----------+---------+
|          date|Usage_kWh|Lagging_Current_Reactive_Power_kVarh|Leading_Current_Reactive_Power_kVarh|CO2(tCO2)|Lagging_Current_Power_Factor|Leading_Current_Power_Factor|  NSM|WeekStatus|Day_of_week|Load_Type|
+--------------+---------+------------------------------------+------------------------------------+---------+----------------------------+----------------------------+-----+----------+-----------+---------+
|2/1/2018 18:45|    40.25|                                8.82|                                 0.5|      0.0|                       97.68|                       99.99|67500|         0|          2|        2|
|5/1/2018 13:00|     8.21|                                 0.0|                               25.81|      0.0|                       100.0|                       30.31|

In [10]:
# Check null counts of columns
null_counts = {col: data.filter(data[col].isNull()).count() for col in data.columns}
print(null_counts)

{'date': 0, 'Usage_kWh': 0, 'Lagging_Current_Reactive_Power_kVarh': 0, 'Leading_Current_Reactive_Power_kVarh': 0, 'CO2(tCO2)': 0, 'Lagging_Current_Power_Factor': 0, 'Leading_Current_Power_Factor': 0, 'NSM': 0, 'WeekStatus': 0, 'Day_of_week': 0, 'Load_Type': 0}


In [11]:
# Create Feature Columns from Dataset based on important features
# Around 42% of data in wip columns are null so we omit it

feature_columns = [
 'Lagging_Current_Reactive_Power_kVarh',
 'Leading_Current_Reactive_Power_kVarh',
 'CO2(tCO2)',
 'Lagging_Current_Power_Factor',
 'Leading_Current_Power_Factor',
 'NSM',
 'WeekStatus',
 'Day_of_week',
 'Load_Type']

In [12]:
# Create assembler
assembler = VectorAssembler(inputCols = feature_columns, outputCol = "features")
assembled_data = assembler.transform(data)

In [13]:
# Split Data into training and test set
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=42)

In [14]:
# Create linear regression model and fit it
linearRegression = LinearRegression(featuresCol='features', labelCol='Usage_kWh')
model = linearRegression.fit(train_data)

In [15]:
# Predict the output based on test data
linearRegressionPredictions = model.transform(test_data)

In [16]:
# Create evaluator to evaluate RMSE
evaluator = RegressionEvaluator(labelCol = "Usage_kWh", predictionCol="prediction", metricName="rmse")

In [17]:
# Calcuate RMSE of the model
RMSE = evaluator.evaluate(linearRegressionPredictions)

In [18]:
# Predict RMSE of the model
print("Root Mean Squared Error (RMSE) on test data:", RMSE)

Root Mean Squared Error (RMSE) on test data: 4.843285121732955


In [19]:
# Create Generalized Linear Regression model and fit it
generalizedLinearRegression = GeneralizedLinearRegression(featuresCol = 'features', labelCol = 'Usage_kWh')
generalizedLinearRegressionModel = generalizedLinearRegression.fit(train_data)

In [20]:
# Predict the output based on test data
generalizedLinearRegressionPredictions =  generalizedLinearRegressionModel.transform(test_data)

In [21]:
# Calcuate RMSE of the model
generalizedLinearRegressionRMSE = evaluator.evaluate(generalizedLinearRegressionPredictions)

In [22]:
# Predict RMSE of the model
print("Root Mean Squared Error (RMSE) on test data:", generalizedLinearRegressionRMSE)

Root Mean Squared Error (RMSE) on test data: 4.843285121732955


In [23]:
# Create Decision Tree Regressor model and fit it
decisionTreeRegression = DecisionTreeRegressor(featuresCol = 'features', labelCol = 'Usage_kWh')
decisionTreeRegressionModel = decisionTreeRegression.fit(train_data)

In [24]:
# Predict the output based on test data
decisionTreeRegressionPredictions =  decisionTreeRegressionModel.transform(test_data)

In [25]:
# Calcuate RMSE of the model
decisionTreeRegressionRMSE = evaluator.evaluate(decisionTreeRegressionPredictions)

In [26]:
# Predict RMSE of the model
print("Root Mean Squared Error (RMSE) on test data:", decisionTreeRegressionRMSE)

Root Mean Squared Error (RMSE) on test data: 3.6215047914826006


In [27]:
# Create Gradient Boosted Regressor model and fit it
gradientBoostedRegression = GBTRegressor(featuresCol = 'features', labelCol = 'Usage_kWh')
gradientBoostedRegressionModel = gradientBoostedRegression.fit(train_data)

In [28]:
# Predict the output based on test data
gradientBoostedRegressionPredictions =  gradientBoostedRegressionModel.transform(test_data)

In [29]:
# Calcuate RMSE of the model
gradientBoostedRegressionRMSE = evaluator.evaluate(gradientBoostedRegressionPredictions)

In [30]:
# Predict RMSE of the model
print("Root Mean Squared Error (RMSE) on test data:", gradientBoostedRegressionRMSE)

Root Mean Squared Error (RMSE) on test data: 2.7656469391292813


In [31]:
# Create Random Forest Regression model and fit it
randomForestRegression = RandomForestRegressor(featuresCol='features', labelCol='Usage_kWh')
randomForestModel = randomForestRegression.fit(train_data)

In [32]:
# Predict the output based on test data
randomForestPredictions = randomForestModel.transform(test_data)

In [33]:
# Calcuate RMSE of the model
randomForestRMSE = evaluator.evaluate(randomForestPredictions)

In [34]:
# Predict RMSE of the model
print("Root Mean Squared Error (RMSE) on test data:", randomForestRMSE)

Root Mean Squared Error (RMSE) on test data: 5.0305284363612675


In [35]:
# Create Cross validator for linear regression model
# Create a parameter grid for hyperparameter tuning
linearRegressionParamGrid = ParamGridBuilder() \
    .addGrid(linearRegression.regParam, [0.1, 0.01]) \
    .addGrid(linearRegression.fitIntercept, [True, False]) \
    .addGrid(linearRegression.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()


# Set up CrossValidator with 5-fold cross-validation
linearRegressionCrossValidation = CrossValidator(
    estimator=linearRegression,
    estimatorParamMaps=linearRegressionParamGrid,
    evaluator=evaluator,
    numFolds=5,
    parallelism=2
)

# Fit the model using CrossValidator
linearRegressionCrossValidationModel = linearRegressionCrossValidation.fit(assembled_data)

In [36]:
# Print average accuracy after cross validation
linearRegressionCrossValidationModel.avgMetrics[0]

4.703554163102085

In [37]:
# Create Generalized Linear Regression model and fit it
generalizedLinearRegression = GeneralizedLinearRegression(featuresCol = 'features', labelCol = 'Usage_kWh')
generalizedLinearRegressionModel = generalizedLinearRegression.fit(train_data)

In [38]:
 # Create Cross validator for generalized linear regression model
# Create a parameter grid for hyperparameter tuning
generalizedLinearRegressionParamGrid = ParamGridBuilder() \
    .addGrid(generalizedLinearRegression.regParam, [0.1, 0.01]) \
    .addGrid(generalizedLinearRegression.fitIntercept, [True, False]) \
    .build()


# Set up CrossValidator with 5-fold cross-validation
generalizedLinearRegressionCrossValidation = CrossValidator(
    estimator=generalizedLinearRegression,
    estimatorParamMaps=generalizedLinearRegressionParamGrid,
    evaluator=evaluator,
    numFolds=5,
    parallelism=2
)

# Fit the model using CrossValidator
generalizedLinearRegressionCrossValidationModel = generalizedLinearRegressionCrossValidation.fit(assembled_data)

In [39]:
# Print average accuracy after cross validation
generalizedLinearRegressionCrossValidationModel.avgMetrics[0]

4.703554163102083

In [40]:
# Create Cross validator for decision tree regressor model
# Create a parameter grid for hyperparameter tuning
decisionTreeRegressionParamGrid = ParamGridBuilder() \
    .addGrid(decisionTreeRegression.maxDepth, [2, 5, 10]) \
    .addGrid(decisionTreeRegression.minInstancesPerNode, [1, 2, 4]) \
    .build()


# Set up CrossValidator with 5-fold cross-validation
decisionTreeRegressionCrossValidation = CrossValidator(
    estimator=decisionTreeRegression,
    estimatorParamMaps=decisionTreeRegressionParamGrid,
    evaluator=evaluator,
    numFolds=5,
    parallelism=2
)

# Fit the model using CrossValidator
decisionTreeRegressionCrossValidationModel = decisionTreeRegressionCrossValidation.fit(assembled_data)

In [41]:
# Print average accuracy after cross validation
decisionTreeRegressionCrossValidationModel.avgMetrics[0]

8.397791706510253

In [42]:
# Create Generalized Linear Regression model and fit it
gradientBoostedRegression = GBTRegressor(featuresCol = 'features', labelCol = 'Usage_kWh')
gradientBoostedRegressionModel = gradientBoostedRegression.fit(train_data)

In [43]:
# Create Cross validator for gradient boosted tree regressor model
# Create a parameter grid for hyperparameter tuning
gradientBoostedRegressionParamGrid = ParamGridBuilder() \
    .addGrid(gradientBoostedRegression.maxDepth, [2, 5, 10]) \
    .addGrid(gradientBoostedRegression.minInstancesPerNode, [1, 2, 4]) \
    .build()


# Set up CrossValidator with 5-fold cross-validation
gradientBoostedRegressionCrossValidation = CrossValidator(
    estimator=gradientBoostedRegression,
    estimatorParamMaps=gradientBoostedRegressionParamGrid,
    evaluator=evaluator,
    numFolds=5,
    parallelism=2
)

# Fit the model using CrossValidator
gradientBoostedRegressionCrossValidationModel = gradientBoostedRegressionCrossValidation.fit(assembled_data)

In [44]:
# Print average accuracy after cross validation
gradientBoostedRegressionCrossValidationModel.avgMetrics[0]

4.48505302154431

In [45]:
# Create Cross validator for random forest regressor model
# Create a parameter grid for hyperparameter tuning
randomForestRegressionParamGrid = ParamGridBuilder() \
    .addGrid(randomForestRegression.numTrees, [10, 20, 50]) \
    .addGrid(randomForestRegression.maxDepth, [2, 5, 10]) \
    .build()


# Set up CrossValidator with 5-fold cross-validation
randomForestRegressionCrossValidation = CrossValidator(
    estimator=randomForestRegression,
    estimatorParamMaps=randomForestRegressionParamGrid,
    evaluator=evaluator,
    numFolds=5,
    parallelism=2
)

# Fit the model using CrossValidator
randomForestRegressionCrossValidationModel = randomForestRegressionCrossValidation.fit(assembled_data)

In [46]:
# Print average accuracy after cross validation
randomForestRegressionCrossValidationModel.avgMetrics[0]

9.592470915311976

In [47]:
# For correlation heatmap
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = 'data.csv'  # Replace with your actual dataset path
data = pd.read_csv(file_path)

# Select numerical columns for correlation
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns

# Calculate the correlation matrix
correlation_matrix = data[numerical_columns].corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()


In [48]:
# Export data

# Export linear regression predictions
results = linearRegressionPredictions.select("Usage_kWh", "prediction").toPandas()
results.to_csv("linearRegression.csv", index = False)

In [49]:
# Export generalized linear regression predictions
results = generalizedLinearRegressionPredictions.select("Usage_kWh", "prediction").toPandas()
results.to_csv("generalizedLinearRegression.csv", index = False)

In [50]:
# Export decision tree regression predictions
results = decisionTreeRegressionPredictions.select("Usage_kWh", "prediction").toPandas()
results.to_csv("decisionTreeRegression.csv", index = False)

In [51]:
# Export gradient boosted regression predictions
results = gradientBoostedRegressionPredictions.select("Usage_kWh", "prediction").toPandas()
results.to_csv("gradientBoostedRegression.csv", index = False)

In [52]:
# Export random forest regressor predictions
results = randomForestPredictions.select("Usage_kWh", "prediction").toPandas()
results.to_csv("randomForestRegression.csv", index = False)